# Zomato Stock Analysis -->>

**Uploding Data**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Uploding Data**

In [2]:
zomato = "/content/drive/MyDrive/Colab Notebooks/ZOMATO.csv"
df = pd.read_csv(zomato)

FileNotFoundError: ignored

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.shape  # Shape Of The Data --->>

In [ ]:
df.describe()

In [ ]:
df.info()  # info of our Dataset --->>

In [ ]:
df = pd.read_csv(zomato,parse_dates=["Date"], index_col="Date")   # Date is object converting into DateTime --->>
df.head(5)

In [ ]:
df.isnull().sum()  # Checking Null Values In Dataset ---->>

# Let's see a historical view of the closing price

In [ ]:
df['Adj Close'].plot()
plt.title("Adj Close")

# Now let's plot the total volume of stock being traded each day

In [ ]:
df['Volume'].plot()
plt.ylabel('Volume')

# MOVING AVERAGE OF ["Adj Close"]


In [ ]:
ma_day = [10, 20, 50]

for ma in ma_day:
  column_name = f"MA for {ma} days"
  df[column_name] = df['Adj Close'].rolling(ma).mean()

In [ ]:
df.columns  # Columns 

In [ ]:
df[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot()
plt.title('Moving Average')


# We'll use pct_change to find the percent change for each day

In [ ]:
df['Daily Return'] = df['Adj Close'].pct_change()

In [ ]:
df.head()

In [ ]:
f = plt.figure()      # Defining Size ---><>
f.set_figwidth(15)
f.set_figheight(8)

df['Daily Return'].plot(legend=True, linestyle='--', marker='o')
plt.title('ZOMATO')

In [ ]:
df['Daily Return'].hist(bins=50)
plt.ylabel('Daily Return')
plt.title("Zomato")

# All Comparision's

In [ ]:
sns.pairplot(df[['Close','Adj Close','Volume','Daily Return']])

# Scaling And Spliting Data --->>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['Open','High','Low','Volume']],df[['Adj Close']] ,test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
scaler = MinM=MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# #Process the data for LSTM
# trainX =np.array(X_train)
# testX =np.array(X_test)
# X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
# X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])


# Convert the x_train and y_train to numpy arrays 
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM


In [ ]:
# #Building the LSTM Model
# lstm = Sequential()
# lstm.add(LSTM(128, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=True))
# lstm.add(LSTM(64, return_sequences=False))
# lstm.add(Dense(25))
# lstm.add(Dense(1))

# lstm.compile(loss='mean_squared_error', optimizer='adam')
# #plot_model(lstm, show_shapes=True, show_layer_names=True)


model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=120, batch_size=8, verbose=1, shuffle=False)

In [ ]:
X_test = np.array(X_test)

# Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))


# Comparision Between Predicted And Original And Creating
# New Dataframe For it 

In [ ]:
predictions = model.predict(X_test)
df1 = pd.DataFrame(predictions)

In [ ]:
df1.rename(columns={0:'prediction'},inplace=True)

In [ ]:
df2 = pd.DataFrame(y_test)

In [ ]:
 df2 = df2.reset_index()

In [ ]:
result = pd.concat([df2, df1], axis=1, join='inner')
display(result)

In [ ]:
from sklearn import metrics 

In [ ]:
metrics.r2_score(predictions,y_test)